# Anlysis of museum collection

### Import all the module required

In [88]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Import the data from the website

In [89]:
collection_df = pd.read_csv("https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv")
collection_df

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository,Tags
0,1979.486.1,False,False,1,American Decorative Arts,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/1,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
1,1980.264.5,False,False,2,American Decorative Arts,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/2,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
2,67.265.9,False,False,3,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/3,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
3,67.265.10,False,False,4,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/4,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
4,67.265.11,False,False,5,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/5,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
5,67.265.12,False,False,6,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/6,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
6,67.265.13,False,False,7,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/7,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",Birds|Coins
7,67.265.14,False,False,8,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/8,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",Inscriptions|Men|Profiles|Coins
8,67.265.15,False,False,9,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/9,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
9,1979.486.3,False,False,10,American Decorative Arts,Coin,Two-and-a-half-dollar Indian Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/10,3/11/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN


## Data clean

The data now looks fine. We intend to do some data clean work as follows:
+ Check "Object Begin Date" and "Object End Date" with "Object Date"
+ Check "Artist Begin Date" and "Artist End Date" with "Artist Display Bio"
+ Check "Artist Display Name" with "Artist Alpha Sort", then delete the later one.
+ Merge "Artist Role" with "Artist Prefix"


In [90]:
## modify the names of columns for indexing convenience
collection_df.columns = ['_'.join(col_name.split()) for col_name in collection_df]

In [ ]:
## Check "Object Begin Date" and "Object End Date" with "Object Date"
def check_Object_Date(row):
    return row.Object_End_Date >= row.Object_Begin_Date

def modify_Object_Date(df):
    test_df = df[df.Object_Date.notna()]
    
    for i in test_df.index:
        obdate = df.Object_Date.loc[i]
        year = re.compile("\d+").findall(obdate)
        assert len(year) > 0
        if "-" in obdate:
            if 'B.C.' in obdate:
                bdate = -int(year[0])*100
                edate = -int(year[1])*100
            elif 'century' in obdate:
                bdate = int(year[0])*100
                edate = int(year[1])*100
            elif len(year[0]) == 4 and len(year[1]) == 4:
                bdate = int(year[0])
                edate = int(year[1])
            elif len(year[0]) == 4 and len(year[1]) == 2:
                bdate = int(year[0])
                edate = bdate//100*100 + int(year[1])
            else:
                print('case1',df.loc[i,['Object_Date','Object_Begin_Date','Object_End_Date']],sep = '\n')
                break
        else:
            if 'B.C.' in obdate:
                bdate = -int(year[0])*100 + 1
                edate = bdate + 99
            elif 'century' in obdate:
                bdate = (int(year[0])-1)*100
                edate = bdate + 99
            elif len(year[0]) == 4:
                bdate = edate = int(year[0])
            else:
                print('case2',df.loc[i,['Object_Date','Object_Begin_Date','Object_End_Date']],sep = '\n')
                break
        if df.loc[i].Object_Begin_Date < bdate:
            df.loc[i].Object_Begin_Date = bdate
        if df.loc[i].Object_End_Date > edate:
            df.loc[i].Object_End_Date = edate
    return df

#     a = test_df[ ["century" in date for date in test_df.Object_Date]][['Object_Date','Object_Begin_Date','Object_End_Date']]
#     return a[-a.apply(check_Object_Date,axis = 1)][['Object_Date','Object_Begin_Date','Object_End_Date']]

modify_Object_Date(collection_df)

In [ ]:
## Check "Object Begin Date" and "Object End Date" with "Object Date"
def check_Object_Date(row):
    return row.Object_End_Date < row.Object_Begin_Date

def modify_Object_Date(df):
    
    return df[df.apply(check_Object_Date,axis = 1)]

modify_Object_Date(collection_df)[['Object_Date','Object_Begin_Date','Object_End_Date']]

In [ ]:
test_df = collection_df[collection_df.Object_Date.notna()]
test_df[["-" in i and len(re.compile("\d+").findall(i)) < 2 for i in test_df.Object_Date]][['Object_Date','Object_Begin_Date','Object_End_Date']]

We intend to check number of NaNs in all the columns first and try to ingnore those columns which the number of NaNs is no more than 10%.

In [ ]:
collection_df.isna().sum()

In [ ]:
collection_df.dtypes

In [ ]:
filtered_df =  collection_df.loc[:, collection_df.isna().sum(axis = 0) < len(collection_df.index)*0.1 ]
filtered_df

## Data analysis & Data Visualization

In [ ]:
## explore the type of object
top_ten_collections = collection_df.Object_Name.value_counts(dropna=False).head(10)
top_ten_collections

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = top_ten_collections.values, y = top_ten_collections.index, alpha=0.8, orient='h')
plt.title('Top Ten Collection Types in The Museum')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Collection Types', fontsize=12)
plt.show()

In [ ]:
## explore the departement
departments = collection_df.Department.value_counts()
departments

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(x = departments.values, y = departments.index, alpha=0.8, orient='h')
plt.title('Quantiles of Collections in Different Departments')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Names of Departments', fontsize=12)
plt.show()

In [ ]:
collection_df.Repository.value_counts()

In [146]:
## explore Credit Line
credit_name = collection_df[collection_df.Credit_Line.notna()]['Credit_Line'].str.split(',').tolist()
# print(credit_name[0][-1])

year_counts = {}
contribute_year = []
# for i in range(len(credit_name)):
#     pattern = re.compile(r'\d{4}')
#     string = ''.join(credit_name[i])
#     contribute_year += re.findall(credit_name[i], string)
#     i += 1
# print(contribute_year)
for ind,ele in enumerate(credit_name):
    contribute_year.append(ele[-1])
# from collection import Counter
for key in range(len(contribute_year)):
    year_counts[key] = year_counts.get(key, 0) + 1
print(year_counts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Analysis conclusion